In [2]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np

In [7]:
def char_to_weight(character):
    if character == "x":
        return np.array([0, 1, 0, 0])
    if character == "y":
        return np.array([0, 0, 1, 0])
    if character == "z":
        return np.array([0, 0, 0, 1])
    if character == "i":
        return np.array([1, 0, 0, 0])
def char_to_index(character):
    if character == "x":
        return 1
    if character == "y":
        return 2
    if character == "z":
        return 3
    if character == "i":
        return 0
    
def group_instructorss_by_qubits(nested_list, n):
    grouped_data = []
    for sublist in nested_list:
        groups = {i: [] for i in range(n)}  # Create empty groups for all indices 0 to n
        for item in sublist:
            key = item[1]  # Second value in the tuple
            groups[key].append(item)
        grouped_data.append([groups[i] for i in range(n)])  # Append the lists in order
    return grouped_data

def mapper_noncx(character, instructors):
    # Map a single Pauliword to list by multiple instructors
    # Example: X -> [0, 1, 0, 0] -- h --> [0,0,-1,0] = -Y
    from numpy import sin, cos, sqrt
    weights = char_to_weight(character)
    for gate, index, param in instructors:
        I, A, B, C = weights
        if gate == "h":
            weights = np.array([I, C, -B, A])
        if gate == "s":
            weights = np.array([I, -B, A, C])
        if gate == "t":
            weights = np.array([I, (A - B) / sqrt(2), (A + B) / sqrt(2), C])
        if gate == "rx":
            weights = np.array([I, A, B * cos(param) - C * sin(param), B * sin(param) + C * cos(param)])
        if gate == "ry":
            weights = np.array([I, A * cos(param) + C * sin(param), B, C * cos(param) - A * sin(param)])
        if gate == "rz":
            weights = np.array([I, A * cos(param) - B * sin(param), B * cos(param) + A * sin(param), C])
    return weights

def construct_LUT_noncx(instructorsss, num_qubits):
    # instructorss has size k x n x [?], with k is number of non-cx layer, n is number of qubits, 
    # ? is the number of instructor.
    # lut has size k x n x 4 x 4, with 4 is the number of Pauliword, 4 for weights
    k = len(instructorsss)
    lut = np.zeros((k, num_qubits, 4, 4))
    print(lut.shape)
    characters = ["i", "x", "y", "z"]
    for k in range(k):
        for j in range(num_qubits):
            for i in range(4):
                lut[k][j][i] = mapper_noncx(characters[i], instructorsss[k][j])
    return lut

class Instructor:
    def __init__(self, num_qubits):
        self.clusters = []
        self.cluster = []
        self.cluster_temp = []
        self.xcluster = []
        self.xcluster_temp = []
        self.xclusters = []
        self.instructors = []
        self.num_qubits = num_qubits
        self.barriers = [0] * self.num_qubits
        self.is_cx_first = False
    def append(self, gate, index, param=0):
        self.instructors.append((gate, index, param))

    def clustering(self):
        if self.instructors[0][0] == "cx":
            self.is_cx_first = True
        self.barriers = [0] * self.num_qubits
        while len(self.instructors) > 0:
            gate, index, _ = self.instructors[0]
            
            is_break = False
            if gate == "cx":
                self.barriers[index[0]] += 1
                self.barriers[index[1]] += 1
                if sum(self.barriers) >= self.num_qubits and np.all(self.barriers):
                    if len(self.instructors) > 1:
                        if self.instructors[1][0] != "cx":
                            is_break = True

                self.xcluster.append(self.instructors.pop(0))
            else:
                if self.barriers[index] == 0:
                    self.cluster.append(self.instructors.pop(0))
                else:
                    self.cluster_temp.append(self.instructors.pop(0))
            if is_break:
                if len(self.cluster) > 0:
                    self.clusters.append(self.cluster)
                self.instructors =  self.cluster_temp + self.instructors
                if len(self.xcluster) > 0:
                    self.xclusters.append(self.xcluster)
                self.cluster = []
                self.cluster_temp = []
                self.xcluster = []
                self.barriers = [0] * self.num_qubits
                is_break = False
        if len(self.cluster) > 0:
            self.clusters.append(self.cluster)
        if len(self.cluster_temp) > 0:
            self.clusters.append(self.cluster_temp)
        if len(self.xcluster) > 0:
            self.xclusters.append(self.xcluster)
        return 


In [16]:
grouped_instructorss = group_instructorss_by_qubits(ins.clusters, ins.num_qubits)
print(len(grouped_instructorss))
print(len(grouped_instructorss[0]))
# lut has size k x n x 4 x 4
LUT = construct_LUT_noncx(grouped_instructorss, ins.num_qubits)

5
3
(5, 3, 4, 4)


In [21]:
words = [['z','i', 'i']]
num_qubits = 3
k = 0
# Init
# 2**n is th
lambdas = np.zeros((2**num_qubits))
weightss = np.zeros((2**num_qubits, num_qubits, 4))
for w, word in enumerate(words):
    for j, char in enumerate(word):
        index = char_to_index(char)
        weightss[w][j] = LUT[k][j][index]

In [26]:
weightss.shape

(8, 3, 4)

In [25]:
weightss[0]

array([[ 0.        ,  0.74383163, -0.00105933,  0.6683662 ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ]])

In [53]:

# Define the input list
weightss = np.array([[[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]]])

def weightss_to_list(weightss: np.ndarray) -> np.ndarray:
    # A sum of transformed word (a matrix 4^n x n x 4) to list
    # Example for a single transformed word: [[1,2,3,4], [1,2,3,4]] -> [1, 2, 3, 4, 2, 4, 6, 8, 3, 6, 9, 12, 4, 8, 12, 16]
    # Example for this function (sum, 2 qubits, 3 term):
    # [[[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]]] -> [ 3.  6.  9. 12.  6. 12. 18. 24.  9. 18. 27. 36. 12. 24. 36. 48.]
    num_qubits = weightss.shape[1]
    lists = np.zeros((4**num_qubits))
    for weights in weightss:
        combinations = np.array(np.meshgrid(*weights)).T.reshape(-1, len(weights))
        lists += np.prod(combinations, axis=1)
    return lists

print(weightss_to_list(weightss))


[ 3.  6.  9. 12.  6. 12. 18. 24.  9. 18. 27. 36. 12. 24. 36. 48.]


In [5]:

ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [ ]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [1, 3], 0]]]

In [13]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.clustering()

In [14]:
ins.clusters

[[('rx', 0, 0.4792542746758992),
  ('ry', 0, 0.7178444895678454),
  ('rz', 0, 0.6672183370132274),
  ('rx', 1, 0.3857606257923717),
  ('ry', 1, 0.47369881398055047),
  ('rz', 1, 0.6356524803973157),
  ('rx', 2, 0.6452497698710842),
  ('ry', 2, 0.05358075830677966),
  ('rz', 2, 0.5552902363142603)],
 [('rx', 0, 0.7508895716449558),
  ('ry', 0, 0.057178737245975775),
  ('rz', 0, 0.9585677579132423),
  ('rx', 1, 0.07390337892847842),
  ('ry', 1, 0.6932453586154507),
  ('rz', 1, 0.0805802781623427),
  ('rx', 2, 0.9825872991886534),
  ('ry', 2, 0.7598410714091143),
  ('rz', 2, 0.4896800116532498)],
 [('rx', 0, 0.483682710074266),
  ('ry', 0, 0.36531156600046577),
  ('rz', 0, 0.9249046303216883),
  ('rx', 1, 0.7128800333121091),
  ('ry', 1, 0.5220985559609234),
  ('rz', 1, 0.4276898755870977),
  ('rx', 2, 0.730625286556946),
  ('ry', 2, 0.18326635869828733),
  ('rz', 2, 0.5948723009116327)],
 [('rx', 0, 0.011420802533366525),
  ('ry', 0, 0.7236273729322115),
  ('rz', 0, 0.26383164372609025),

In [15]:
ins.xclusters

[[('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)]]